<a href="https://colab.research.google.com/github/DanielFigueiredo1992/Pip-H2OAutoML-/blob/main/obtedor_de_preco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import json
from google.colab import output
from datetime import datetime, timedelta  # Import timedelta class

# Python function to receive data from JavaScript and construct a pandas DataFrame
def receive_data(data_json):
    global df  # Declare df as global to make it accessible in all cells

    # Convert JSON string to a Python object
    try:
        data = json.loads(data_json)
        print("JSON loaded successfully")
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return

    # Print a sample of the loaded data
    print("First item of data:", data[0] if data else "No data")

    # Check if data is a list
    if not isinstance(data, list):
        print("Data is not a list")
        return

    # Check if each item in data is a list
    for idx, item in enumerate(data):
        if not isinstance(item, list):
            print(f"Item at index {idx} in data is not a list: {item}")
            return

    # Check if each item has the correct length
    expected_length = 12
    for idx, item in enumerate(data):
        if len(item) != expected_length:
            print(f"Item at index {idx} does not have the expected length of {expected_length}: {item}")
            return

    # Convert timestamps to human-readable format
    for i in range(len(data)):
        try:
            data[i][0] = pd.to_datetime(data[i][0], unit='ms')  # Convert open_time
            data[i][6] = pd.to_datetime(data[i][6], unit='ms')  # Convert close_time
        except Exception as e:
            print(f"Error converting timestamps at index {i}: {e}")
            return

    # Print a sample of the data after timestamp conversion
    print("Data after timestamp conversion:", data[:3])

    # Convert the data to a pandas DataFrame
    try:
        df = pd.DataFrame(data, columns=[
            'open_time', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'number_of_trades',
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        print("DataFrame created successfully")
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        return

    # Display the DataFrame

# Register the Python function to be called from JavaScript
output.register_callback('retornoDados', receive_data)

# Define custom initial datetime
initial_datetime = datetime(2024, 5, 1, 0, 0, 0)

# Define final datetime as current date and time in UTC
final_datetime = datetime.utcnow()

# Convert initial and final datetimes to epoch timestamps with milliseconds
initial_epoch = int(initial_datetime.timestamp() * 1000)
final_epoch = int(final_datetime.timestamp() * 1000)

# Calculate the number of intervals (each with 500 records)
num_intervals = (final_epoch - initial_epoch) // (500 * 60000)  # 60000 milliseconds in 1 minute
API_KEY = "m5gNycphXSKHEUT3yCOs875VqXzEGSudt1RGpML9O3PkxVCVNUODIUMiBe8PaTTi";
API_SECRET = "PRgpBJLfzWgnuyZ9OjfA2ODJk8v0YLyhyANXgtzYeud4b6dBGjMzM7EQDK4b5g0o";
#&apiKey=${API_KEY}&apiSecret=${API_SECRET}
# JavaScript code to call the Binance API and retrieve data for the BTCUSDT pair
javascript_code = f"""
async function getData(initialEpoch, finalEpoch, numIntervals) {{
    let allData = [];

    // Define the start time for each interval
    let startTime = initialEpoch;

    // Define the end time for each interval (incremented by 500 records each time)
    let endTime = initialEpoch + 500 * 60000;  // 60000 milliseconds in 1 minute

    // Fetch data for each interval
    for (let i = 0; i <= numIntervals; i++) {{
        // Call the Binance API to get data for the BTCUSDT pair for the current interval


        let url = `https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=1m&startTime=${{startTime}}&endTime=${{endTime}}`;


        // Fetch data from the API
        let response = await fetch(url);
        let data = await response.json();

        // Add the data to the array
        allData.push(...data);

        // Update start and end times for the next interval
        startTime = endTime;
        endTime += 500 * 60000;
    }}

    // Send all the data to Python
    google.colab.kernel.invokeFunction('retornoDados', [JSON.stringify(allData)]);
}}

// Call the getData function with initial and final epochs as parameters
getData({initial_epoch}, {final_epoch}, {num_intervals});
"""

# Register the JavaScript function
output.eval_js(javascript_code)
df

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2024-05-01 00:00:00,60672.01000000,60750.00000000,60660.98000000,60742.33000000,55.30759000,2024-05-01 00:00:59.999,3357946.71253730,1507,38.79723000,2355475.03909240,0
1,2024-05-01 00:01:00,60742.33000000,60779.52000000,60721.76000000,60757.86000000,15.62630000,2024-05-01 00:01:59.999,949322.85073970,772,7.81598000,474852.77969140,0
2,2024-05-01 00:02:00,60757.87000000,60757.87000000,60675.20000000,60678.13000000,38.57373000,2024-05-01 00:02:59.999,2342728.55776130,978,3.34720000,203264.45587220,0
3,2024-05-01 00:03:00,60678.13000000,60709.96000000,60675.21000000,60709.96000000,9.00795000,2024-05-01 00:03:59.999,546737.61109370,788,6.39701000,388261.40982850,0
4,2024-05-01 00:04:00,60709.96000000,60728.00000000,60688.39000000,60688.39000000,27.67657000,2024-05-01 00:04:59.999,1680118.95322200,841,4.94077000,299943.14584880,0
...,...,...,...,...,...,...,...,...,...,...,...,...
28627,2024-05-20 21:07:00,69271.02000000,69286.75000000,69234.51000000,69235.72000000,31.27638000,2024-05-20 21:07:59.999,2166207.33095760,1263,11.50256000,796694.59235680,0
28628,2024-05-20 21:08:00,69235.72000000,69245.62000000,69086.95000000,69125.86000000,92.58436000,2024-05-20 21:08:59.999,6403001.43950590,4108,33.28693000,2302164.97259960,0
28629,2024-05-20 21:09:00,69125.86000000,69259.86000000,69079.24000000,69259.85000000,77.24846000,2024-05-20 21:09:59.999,5341949.57284980,2452,51.97922000,3595157.66654110,0
28630,2024-05-20 21:10:00,69259.85000000,69280.00000000,69180.00000000,69201.06000000,109.61865000,2024-05-20 21:10:59.999,7588715.48636530,2491,47.13643000,3263257.26598270,0


JSON loaded successfully
First item of data: [1714521600000, '60672.01000000', '60750.00000000', '60660.98000000', '60742.33000000', '55.30759000', 1714521659999, '3357946.71253730', 1507, '38.79723000', '2355475.03909240', '0']
Data after timestamp conversion: [[Timestamp('2024-05-01 00:00:00'), '60672.01000000', '60750.00000000', '60660.98000000', '60742.33000000', '55.30759000', Timestamp('2024-05-01 00:00:59.999000'), '3357946.71253730', 1507, '38.79723000', '2355475.03909240', '0'], [Timestamp('2024-05-01 00:01:00'), '60742.33000000', '60779.52000000', '60721.76000000', '60757.86000000', '15.62630000', Timestamp('2024-05-01 00:01:59.999000'), '949322.85073970', 772, '7.81598000', '474852.77969140', '0'], [Timestamp('2024-05-01 00:02:00'), '60757.87000000', '60757.87000000', '60675.20000000', '60678.13000000', '38.57373000', Timestamp('2024-05-01 00:02:59.999000'), '2342728.55776130', 978, '3.34720000', '203264.45587220', '0']]
DataFrame created successfully
